# Preparing the environement

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 132 kB 67.2 MB/s 
     |████████████████████████████████| 133 kB 68.6 MB/s 
     |████████████████████████████████| 47 kB 6.0 MB/s 
     |████████████████████████████████| 226 kB 71.3 MB/s 
     |████████████████████████████████| 135 kB 71.6 MB/s 
     |████████████████████████████████| 29.2 MB 1.3 MB/s 
     |████████████████████████████████| 79.9 MB 121 kB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 48.3 MB 1.5 MB/s 
     |████████████████████████████████| 508 kB 60.6 MB/s 
     |████████████████████████████████| 13.2 MB 52.8 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 106 kB 71.1 MB/s 
     |████████████████████████████████| 563 kB 65.1 MB/s 
     |███████████████████████████████

In [ ]:
# Downgrade torch to accommodate Yolact
!pip install torchvision==0.5.0
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 753.4 MB 7.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uninstalling torchvision-0.13.0+cu113:
      Successfully uninstalled torchvision-0.13.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.4.0 which is incompatible.
fastai 2.7.7 requires torch<1.13,>=1.7, but you have torch 1.4.

In [ ]:
!pip uninstall opencv-python --y
!pip uninstall opencv-contrib-python --y
!pip install opencv-contrib-python==4.5.5.62

**!! Restart the notbook to apply changes, if needed**

# Downloading pre-trained weight and dataset

In [ ]:
import os
import gdown
import json
import pandas as pd
import numpy as np

In [3]:
# Copy S3
!aws s3 cp --recursive s3://aqc-technology-qualitex-clahe/mask-rcnn/Sagemaker/input/train/ s3://aqc-technology-qualitex-clahe/yolact/input/train/

# Train **YOLACT** model

In [ ]:
# Download trainining weight
training_weight = './weights/resnet50-19c8e357.pth'
if not os.path.exists(training_weight):
  os.mkdir('./weights/')
  gdown.download(id='1yp7ZbbDwvMiFJEq4ptVKTYTI2VeRDXl0', output=training_weight, quiet=False)
  

In [ ]:
!python ./train.py --help

usage: train.py [-h] [--batch_size BATCH_SIZE] [--resume RESUME]
                [--start_iter START_ITER] [--num_workers NUM_WORKERS]
                [--cuda CUDA] [--lr LR] [--momentum MOMENTUM] [--decay DECAY]
                [--gamma GAMMA] [--save_folder SAVE_FOLDER]
                [--log_folder LOG_FOLDER] [--config CONFIG]
                [--save_interval SAVE_INTERVAL]
                [--validation_size VALIDATION_SIZE]
                [--validation_epoch VALIDATION_EPOCH] [--keep_latest]
                [--keep_latest_interval KEEP_LATEST_INTERVAL]
                [--dataset DATASET] [--no_log] [--log_gpu] [--no_interrupt]
                [--batch_alloc BATCH_ALLOC] [--no_autoscale]

Yolact Training Script

optional arguments:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        Batch size for training
  --resume RESUME       Checkpoint state_dict file to resume training from. If
                        this is "interrupt",

In [ ]:
!python ./train.py --batch_size=8 --config=yolact_resnet50_qualitex_config --num_workers=0 \
--validation_size=200 --validation_epoch=100

Scaling parameters by 0.25 to account for a batch size of 2.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Initializing weights...
Begin training!

/content/gdrive/MyDrive/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
[  0]       0 || B: 3.188 | C: 6.299 | M: 7.413 | S: 1.948 | T: 18.848 || ETA: 13:46:06 || timer: 1.549
[  0]      10 || B: 15.627 | C: 5.845 | M: 7.995 | S: 1.850 | T: 31.318 || ETA: 10:12:49 || timer: 0.556
[  0]      20 || B: 17.846 | C: 5.714 | M: 8.182 | S: 1.729 | 

In [ ]:
!python eval.py --help

usage: eval.py [-h] [--trained_model TRAINED_MODEL] [--top_k TOP_K]
               [--cuda CUDA] [--fast_nms FAST_NMS]
               [--cross_class_nms CROSS_CLASS_NMS]
               [--display_masks DISPLAY_MASKS]
               [--display_bboxes DISPLAY_BBOXES] [--display_text DISPLAY_TEXT]
               [--display_scores DISPLAY_SCORES] [--display] [--shuffle]
               [--ap_data_file AP_DATA_FILE] [--resume]
               [--max_images MAX_IMAGES] [--output_coco_json]
               [--bbox_det_file BBOX_DET_FILE] [--mask_det_file MASK_DET_FILE]
               [--config CONFIG] [--output_web_json]
               [--web_det_path WEB_DET_PATH] [--no_bar]
               [--display_lincomb DISPLAY_LINCOMB] [--benchmark] [--no_sort]
               [--seed SEED] [--mask_proto_debug] [--no_crop] [--image IMAGE]
               [--images IMAGES] [--video VIDEO]
               [--video_multiframe VIDEO_MULTIFRAME]
               [--score_threshold SCORE_THRESHOLD] [--dataset DATASE

In [ ]:
# Evaluation
!python eval.py --trained_model=./weights/yolact_plus_resnet50_santex_55_16000.pth --score_threshold=0.1 \
--top_k=5  --output_coco_json \
--config=yolact_resnet50_santex_config

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading model... Done.

Processing Images  ██████████████████░░░░░░░░░░░░    174 /    290 (60.00%)    20.04 fps        Warning: Augmentation output an example with no ground truth. Resampling...
Processing Images  ██████████████████████████████    290 /    290 (100.00%)    20.13 fps        
Dumping detections...


In [ ]:
!python eval.py --trained_model=./weights/yolact_plus_resnet50_santex_55_16000.pth --score_threshold=0.1 \
--top_k=5  --images=../valid/:./results/ \
--config=yolact_resnet50_santex_config

Loading model... Done.

/gdrive/MyDrive/santex_rimar/small_dataset/9_F17_R56--R70_0_18_3.png -> ./results/9_F17_R56--R70_0_18_3.png
/gdrive/MyDrive/santex_rimar/small_dataset/5_F17_R140--R154_3_25_1.png -> ./results/5_F17_R140--R154_3_25_1.png
/gdrive/MyDrive/santex_rimar/small_dataset/5_F17_R140--R154_4_29_3.png -> ./results/5_F17_R140--R154_4_29_3.png
/gdrive/MyDrive/santex_rimar/small_dataset/3_F17_R126--R140_4_22_1.png -> ./results/3_F17_R126--R140_4_22_1.png
/gdrive/MyDrive/santex_rimar/small_dataset/2_F17_R112--R126_3_18_2.png -> ./results/2_F17_R112--R126_3_18_2.png
/gdrive/MyDrive/santex_rimar/small_dataset/12_F17_R98--R112_6_23_1.png -> ./results/12_F17_R98--R112_6_23_1.png
/gdrive/MyDrive/santex_rimar/small_dataset/10_F17_R70--R84_2_10_0.png -> ./results/10_F17_R70--R84_2_10_0.png
/gdrive/MyDrive/santex_rimar/small_dataset/5_F17_R140--R154_4_26_2.png -> ./results/5_F17_R140--R154_4_26_2.png
/gdrive/MyDrive/santex_rimar/small_dataset/11_F17_R84--R98_5_12_2.png -> ./results/11_